In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies_unmodified = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1") #unmodified names
companies = pd.read_csv('List_of_companies_Full.csv', encoding = "ISO-8859-1") #will have modified names
current_list = pd.read_csv('official_ratio_with_industries_396.csv', encoding = "ISO-8859-1")

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
print(len(companies_unmodified))
current_list

500
500


,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio,GICS Sector,GICS Sub-Industry
0,3M Company,Michael F. Roman,18321566,57494,319,Industrials,Industrial Conglomerates
1,Abbott Laboratories,na,-1,-1,-1,Health Care,Health Care Equipment
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139,Health Care,Pharmaceuticals
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148,Health Care,Health Care Equipment
4,Accenture plc,na,-1,-1,-1,Information Technology,IT Consulting & Other Services
5,Activision Blizzard,Robert A. Kotick,30122896,94308,319,Communication Services,Interactive Home Entertainment
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266,Information Technology,Application Software
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604,Information Technology,Semiconductors
8,Advance Auto Parts,Thomas R. Greco,7691513,19463,395,Consumer Discretionary,Automotive Retail
9,AES Corp,na,-1,-1,-1,Utilities,Independent Power Producers & Energy Traders


In [3]:
companies_left = ['Abbott Laboratories',
 'Accenture plc',
 'AES Corp',
 'Alexandria Real Estate Equities',
 'Amcor plc',
 'American Tower Corp.',
 'Analog Devices, Inc.',
 'A.O. Smith Corp',
 'Baker Hughes Co',
 'The Bank of New York Mellon',
 'Boeing Company',
 'Broadridge Financial Solutions',
 'Carnival Corp.',
 'Carrier Global',
 'Charles Schwab Corporation',
 'Charter Communications',
 'Chubb Limited',
 'The Clorox Company',
 'The Cooper Companies',
 'Discover Financial Services',
 'eBay Inc.',
 "Edison Int'l",
 'Estée Lauder Companies',
 'Expedia Group',
 'Expeditors',
 'Fidelity National Information Services',
 'First Republic Bank',
 'Fox Corporation',
 'Gap Inc.',
 'Globe Life Inc.',
 'Grainger (W.W.) Inc.',
 'Hartford Financial Svc.Gp.',
 'Healthpeak Properties',
 'The Hershey Company',
 "Honeywell Int'l Inc.",
 'Howmet Aerospace',
 'Huntington Ingalls Industries',
 'IDEXX Laboratories',
 'International Business Machines',
 'International Paper',
 'Interpublic Group',
 'International Flavors & Fragrances',
 'IQVIA Holdings Inc.',
 'Iron Mountain Incorporated',
 'Jack Henry & Associates',
 'J. B. Hunt Transport Services',
 'JM Smucker',
 'KLA Corporation',
 'Laboratory Corp. of America Holding',
 'Lumen Technologies',
 'LyondellBasell',
 'M&T Bank Corp.',
 'MarketAxess',
 'Marsh & McLennan',
 'Mettler Toledo',
 'Mid-America Apartments',
 'Molson Coors Beverage Company',
 'The Mosaic Company',
 'Newmont Corporation',
 'News Corp.',
 'Nike, Inc.',
 'NortonLifeLock',
 'Norwegian Cruise Line Holdings',
 'Old Dominion Freight Line',
 'Otis Worldwide',
 'PACCAR Inc.',
 'Paycom',
 'PayPal',
 'Perrigo',
 'Pioneer Natural Resources',
 'PNC Financial Services',
 'Principal Financial Group',
 'Progressive Corp.',
 'Public Service Enterprise Group (PSEG)',
 'PVH Corp.',
 'Raymond James Financial Inc.',
 'Raytheon Technologies',
 'Robert Half International',
 'Rockwell Automation Inc.',
 'Royal Caribbean Group',
 'Salesforce.com',
 'SVB Financial',
 'Synopsys Inc.',
 'T. Rowe Price Group',
 'Take-Two Interactive',
 'Teledyne Technologies',
 'TJX Companies Inc.',
 'Tractor Supply Company',
 'Trane Technologies plc',
 'The Travelers Companies Inc.',
 'Truist Financial',
 'Tyson Foods',
 'U.S. Bancorp',
 'Universal Health Services',
 'ViacomCBS',
 'Vontier',
 'W. R. Berkley Corporation',
 'Westinghouse Air Brake Technologies Corp',
 'Wells Fargo',
 'West Pharmaceutical Services',
 'Williams Companies',
 'Zebra Technologies',
 'Zimmer Biomet',
 'Zions Bancorp']
companies_left_index = [1,
 4,
 9,
 16,
 25,
 31,
 38,
 42,
 59,
 62,
 70,
 77,
 86,
 87,
 98,
 99,
 102,
 111,
 124,
 144,
 160,
 162,
 172,
 178,
 179,
 187,
 190,
 200,
 203,
 212,
 215,
 218,
 221,
 223,
 230,
 233,
 237,
 239,
 247,
 248,
 249,
 250,
 255,
 256,
 257,
 259,
 260,
 272,
 277,
 292,
 293,
 294,
 297,
 299,
 310,
 315,
 317,
 322,
 331,
 332,
 335,
 340,
 341,
 348,
 352,
 353,
 357,
 358,
 363,
 368,
 369,
 373,
 375,
 378,
 381,
 387,
 388,
 395,
 396,
 400,
 402,
 421,
 423,
 426,
 427,
 432,
 439,
 440,
 441,
 443,
 444,
 447,
 450,
 457,
 467,
 469,
 472,
 473,
 480,
 482,
 488,
 496,
 497,
 498]

In [4]:
col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', [])]

In [5]:
#utilities
def getUrl(n, mod = ''):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + mod + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getIncUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-inc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCoUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-co' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCorpUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-corp' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getPlcUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-plc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getContent(page):
    doc=lh.fromstring(page.content)
    tr_elements=doc.xpath('//tr')
    return tr_elements
def failureCase(col, good, alt_tr_elements):
    for j in range(0,len(alt_tr_elements)):
        #T is our j'th row
        T=alt_tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                T=alt_tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                return True

        return False

In [6]:
#goes through all the webpages
for n in range(len(companies_left_index)):
    print(companies_left_index[n])
    currName = companies.at[companies_left_index[n], 'Security']
    companies_unmodified_names = companies_unmodified.at[companies_left_index[n], 'Security']
    col[0][1].append(companies_unmodified_names)
    #getting url
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    page=requests.get(url)
    #getting all the tables
    tr_elements=getContent(page)
    #indicates that we failed to find the magics on this particular page
    fail = True
    #this will find the table that has the relevant info for the CEO,and put the correct info into col for this one page
    for j in range(0,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                #print(currName1 + ' good')
                #setting fail to false
                fail = False
                T=tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    column_name = col[i + 1][0]
                    current_list.set_value(companies_left_index[n], column_name, data)
                    #Increment i for the next column
                    i+=1
                    
                #col[0][1].append(currName1)
                break
    if fail:
        print(url)
    

1
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
4
https://www1.salary.com/accenture-plc-Executive-Salaries.html
9


C:\Users\larry\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


16
https://www1.salary.com/alexandria-real-estate-equities-Executive-Salaries.html
25
31
38
42
59
62
70
https://www1.salary.com/boeing-co-Executive-Salaries.html
77
86
87
https://www1.salary.com/carrier-global-Executive-Salaries.html
98
99
102
111
124
144
160
https://www1.salary.com/ebay-inc-Executive-Salaries.html
162
172
178
https://www1.salary.com/expedia-group-inc-Executive-Salaries.html
179
187
190
https://www1.salary.com/first-republic-bank-Executive-Salaries.html
200
203
https://www1.salary.com/gap-inc-Executive-Salaries.html
212
https://www1.salary.com/torchmark-corp-Executive-Salaries.html
215
218
221
223
230
233
237
239
247
248
249
250
255
https://www1.salary.com/iqvia-holdings-inc-Executive-Salaries.html
256
257
259
260
272
277
292
https://www1.salary.com/lumen-technologies-Executive-Salaries.html
293
294
297
299
310
315
317
322
331
332
https://www1.salary.com/news-corp-Executive-Salaries.html
335
340
341
348
352
https://www1.salary.com/otis-worldwide-Executive-Salaries.html

In [7]:
df = pd.DataFrame(index=['A','B','C'], columns=['x','y'])
df

,x,y
A,NaN,NaN
B,NaN,NaN
C,NaN,NaN


In [8]:
df.set_value('C', 'x', 10)

C:\Users\larry\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


,x,y
A,NaN,NaN
B,NaN,NaN
C,10,NaN


In [9]:
df

,x,y
A,NaN,NaN
B,NaN,NaN
C,10,NaN


In [10]:
currName = companies.at[n, 'Security']
currName

'church-and-dwight'

In [11]:
current_list

,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio,GICS Sector,GICS Sub-Industry
0,3M Company,Michael F. Roman,18321566,57494,319,Industrials,Industrial Conglomerates
1,Abbott Laboratories,na,-1,-1,-1,Health Care,Health Care Equipment
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139,Health Care,Pharmaceuticals
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148,Health Care,Health Care Equipment
4,Accenture plc,na,-1,-1,-1,Information Technology,IT Consulting & Other Services
5,Activision Blizzard,Robert A. Kotick,30122896,94308,319,Communication Services,Interactive Home Entertainment
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266,Information Technology,Application Software
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604,Information Technology,Semiconductors
8,Advance Auto Parts,Thomas R. Greco,7691513,19463,395,Consumer Discretionary,Automotive Retail
9,AES Corp,Andrés Gluski,10827886,61975,175,Utilities,Independent Power Producers & Energy Traders


In [12]:
int = []
for i in range(len(current_list)):
    if (current_list.at[i, 'Median Employee Pay'] == -1):
        int.append(i)
print(len(int))
int

20


[1,
 4,
 16,
 70,
 87,
 160,
 178,
 190,
 203,
 212,
 255,
 292,
 332,
 352,
 353,
 368,
 396,
 402,
 423,
 432]

In [13]:
current_list.at[16, 'Company Name']

'Alexandria Real Estate Equities'

In [15]:
scraped = []
for i in range(len(current_list)):
    scraped.append("yes")
current_list['Scraped?'] = scraped
for i in range(len(int)):
    current_list.set_value(int[i], 'Scraped?', "no")
current_list

C:\Users\larry\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio,GICS Sector,GICS Sub-Industry,Scraped?
0,3M Company,Michael F. Roman,18321566,57494,319,Industrials,Industrial Conglomerates,yes
1,Abbott Laboratories,na,-1,-1,-1,Health Care,Health Care Equipment,no
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139,Health Care,Pharmaceuticals,yes
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148,Health Care,Health Care Equipment,yes
4,Accenture plc,na,-1,-1,-1,Information Technology,IT Consulting & Other Services,no
5,Activision Blizzard,Robert A. Kotick,30122896,94308,319,Communication Services,Interactive Home Entertainment,yes
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266,Information Technology,Application Software,yes
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604,Information Technology,Semiconductors,yes
8,Advance Auto Parts,Thomas R. Greco,7691513,19463,395,Consumer Discretionary,Automotive Retail,yes
9,AES Corp,Andrés Gluski,10827886,61975,175,Utilities,Independent Power Producers & Energy Traders,yes


In [20]:
current_list.to_csv('official_ratio_with_industries_480.csv',index=False)